0) Set tf random seed, fix typos in preprocess.

1) Based on Version 6 of https://www.kaggle.com/stanleyjzheng/baseline-nn-with-k-folds kernel.

2) Add 7-MultilabelStratifiedKFold, 7 seeds averaging. 

3) Add WeightNormalization, Lookahead, ReduceLROnPlateau.

4) Put zeros for `ctl_vehicle` predictions.

5) Remove control group from training and validation.

6) Model checkpoints (from https://www.kaggle.com/ravy101/drug-moa-tf-keras-starter kernel)

7) Small architecture tune (lb - near noise effect).

8) Feature selection (on full validation): top features by permutation importance (lb - same as 7).


### What doesn't work* for me:
* pretrain on nonscored targets
* training on both scored and nonscored targets
* RAdam, AdamW
* weighted loss, focal loss (probably my mistake)
* selu, leaky-relu
* pseudolabeling ([this](https://www.kaggle.com/simakov/keras-multilabel-neural-network-v1-2?scriptVersionId=42045646) version)

*based on local cross validation.
### In future versions:
* ...

In [1]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [3]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')

ss = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [4]:
def preprocess(df):
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    del df['sig_id']
    return df

train = preprocess(train_features)
test = preprocess(test_features)

del train_targets['sig_id']

train_targets = train_targets.loc[train['cp_type']==0].reset_index(drop=True)
train = train.loc[train['cp_type']==0].reset_index(drop=True)

In [5]:
def create_model(num_columns):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(num_columns),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid"))
    ])
    model.compile(optimizer=tfa.optimizers.Lookahead(tf.optimizers.Adam(), sync_period=10),
                  loss='binary_crossentropy', 
                  )
    return model

In [6]:
'''
from typing import Tuple, List, Callable, Any

from sklearn.utils import check_random_state  # type: ignore

### from eli5
def iter_shuffled(X, columns_to_shuffle=None, pre_shuffle=False,
                  random_state=None):
    rng = check_random_state(random_state)

    if columns_to_shuffle is None:
        columns_to_shuffle = range(X.shape[1])

    if pre_shuffle:
        X_shuffled = X.copy()
        rng.shuffle(X_shuffled)

    X_res = X.copy()
    for columns in tqdm(columns_to_shuffle):
        if pre_shuffle:
            X_res[:, columns] = X_shuffled[:, columns]
        else:
            rng.shuffle(X_res[:, columns])
        yield X_res
        X_res[:, columns] = X[:, columns]



def get_score_importances(
        score_func,  # type: Callable[[Any, Any], float]
        X,
        y,
        n_iter=5,  # type: int
        columns_to_shuffle=None,
        random_state=None
    ):
    rng = check_random_state(random_state)
    base_score = score_func(X, y)
    scores_decreases = []
    for i in range(n_iter):
        scores_shuffled = _get_scores_shufled(
            score_func, X, y, columns_to_shuffle=columns_to_shuffle,
            random_state=rng, base_score=base_score
        )
        scores_decreases.append(scores_shuffled)

    return base_score, scores_decreases



def _get_scores_shufled(score_func, X, y, base_score, columns_to_shuffle=None,
                        random_state=None):
    Xs = iter_shuffled(X, columns_to_shuffle, random_state=random_state)
    res = []
    for X_shuffled in Xs:
        res.append(-score_func(X_shuffled, y) + base_score)
    return res

def metric(y_true, y_pred):
    metrics = []
    for i in range(y_pred.shape[1]):
        if y_true[:, i].sum() > 1:
            metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float)))
    return np.mean(metrics)   

perm_imp = np.zeros(train.shape[1])
all_res = []
for n, (tr, te) in enumerate(KFold(n_splits=7, random_state=0, shuffle=True).split(train_targets)):
    print(f'Fold {n}')

    model = create_model(len(train.columns))
    checkpoint_path = f'repeat:{seed}_Fold:{n}.hdf5'
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
    cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
    model.fit(train.values[tr],
                  train_targets.values[tr],
                  validation_data=(train.values[te], train_targets.values[te]),
                  epochs=35, batch_size=128,
                  callbacks=[reduce_lr_loss, cb_checkpt], verbose=2
                 )
        
    model.load_weights(checkpoint_path)
        
    def _score(X, y):
        pred = model.predict(X)
        return metric(y, pred)

    base_score, local_imp = get_score_importances(_score, train.values[te], train_targets.values[te], n_iter=1, random_state=0)
    all_res.append(local_imp)
    perm_imp += np.mean(local_imp, axis=0)
    print('')
    
top_feats = np.argwhere(perm_imp < 0).flatten()
top_feats
'''

"\nfrom typing import Tuple, List, Callable, Any\n\nfrom sklearn.utils import check_random_state  # type: ignore\n\n### from eli5\ndef iter_shuffled(X, columns_to_shuffle=None, pre_shuffle=False,\n                  random_state=None):\n    rng = check_random_state(random_state)\n\n    if columns_to_shuffle is None:\n        columns_to_shuffle = range(X.shape[1])\n\n    if pre_shuffle:\n        X_shuffled = X.copy()\n        rng.shuffle(X_shuffled)\n\n    X_res = X.copy()\n    for columns in tqdm(columns_to_shuffle):\n        if pre_shuffle:\n            X_res[:, columns] = X_shuffled[:, columns]\n        else:\n            rng.shuffle(X_res[:, columns])\n        yield X_res\n        X_res[:, columns] = X[:, columns]\n\n\n\ndef get_score_importances(\n        score_func,  # type: Callable[[Any, Any], float]\n        X,\n        y,\n        n_iter=5,  # type: int\n        columns_to_shuffle=None,\n        random_state=None\n    ):\n    rng = check_random_state(random_state)\n    base_sco

In [7]:
top_feats = [  1,   2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
        32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
       198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
       254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
       267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
       295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
       310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
       324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
       337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
       350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
       392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
       405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
       419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
       432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
       463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
       506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
       522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
       538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
       552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
       571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
       586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
       600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
       618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
       631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
       645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
       660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
       673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
       686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
       701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
       718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
       733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
       748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
       762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
       775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
       789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
       804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
       821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
       837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
       854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
       870, 871, 872, 873, 874]
print(len(top_feats))

785


In [8]:
def metric(y_true, y_pred):
    metrics = []
    for _target in train_targets.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)

In [9]:
N_STARTS = 7
tf.random.set_seed(42)

res = train_targets.copy()
ss.loc[:, train_targets.columns] = 0
res.loc[:, train_targets.columns] = 0

for seed in range(N_STARTS):
    for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits=7, random_state=seed, shuffle=True).split(train_targets, train_targets)):
        print(f'Fold {n}')
    
        model = create_model(len(top_feats))
        checkpoint_path = f'repeat:{seed}_Fold:{n}.hdf5'
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
        cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
        model.fit(train.values[tr][:, top_feats],
                  train_targets.values[tr],
                  validation_data=(train.values[te][:, top_feats], train_targets.values[te]),
                  epochs=35, batch_size=128,
                  callbacks=[reduce_lr_loss, cb_checkpt], verbose=2
                 )
        
        model.load_weights(checkpoint_path)
        test_predict = model.predict(test.values[:, top_feats])
        val_predict = model.predict(train.values[te][:, top_feats])
        
        ss.loc[:, train_targets.columns] += test_predict
        res.loc[te, train_targets.columns] += val_predict
        print('')
    
ss.loc[:, train_targets.columns] /= ((n+1) * N_STARTS)
res.loc[:, train_targets.columns] /= N_STARTS

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Fold 0
Epoch 1/35
147/147 - 2s - loss: 0.6058 - val_loss: 0.3269
Epoch 2/35
147/147 - 1s - loss: 0.1576 - val_loss: 0.0627
Epoch 3/35
147/147 - 1s - loss: 0.0472 - val_loss: 0.0331
Epoch 4/35
147/147 - 1s - loss: 0.0305 - val_loss: 0.0253
Epoch 5/35
147/147 - 1s - loss: 0.0254 - val_loss: 0.0223
Epoch 6/35
147/147 - 1s - loss: 0.0228 - val_loss: 0.0211
Epoch 7/35
147/147 - 1s - loss: 0.0218 - val_loss: 0.0201
Epoch 8/35
147/147 - 1s - loss: 0.0206 - val_loss: 0.0195
Epoch 9/35
147/147 - 1s - loss: 0.0202 - val_loss: 0.0191
Epoch 10/35
147/147 - 1s - loss: 0.0198 - val_loss: 0.0187
Epoch 11/35
147/147 - 1s - loss: 0.0192 - val_loss: 0.0184
Epoch 12/35
147/147 - 1s - loss: 0.0188 - val_loss: 0.0182
Epoch 13/35
147/147 - 1s - loss: 0.0184 - val_loss: 0.0180
Epoch 14/35
147/147 - 1s - loss: 0.0183 - val_loss: 0.0177
Epoch 15/35
147/147 - 2s - loss: 0.0179 - val_loss: 0.0175
Epoch 16/35
147/147 - 1s - loss: 0.0178 - val_loss: 0.0174
Epoch 17/35
147/147 - 2s - loss: 0.0176 - val_loss: 0.0172

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Fold 0
Epoch 1/35
147/147 - 2s - loss: 0.6061 - val_loss: 0.3073
Epoch 2/35
147/147 - 1s - loss: 0.1572 - val_loss: 0.0621
Epoch 3/35
147/147 - 1s - loss: 0.0473 - val_loss: 0.0334
Epoch 4/35
147/147 - 2s - loss: 0.0304 - val_loss: 0.0250
Epoch 5/35
147/147 - 1s - loss: 0.0252 - val_loss: 0.0225
Epoch 6/35
147/147 - 1s - loss: 0.0233 - val_loss: 0.0216
Epoch 7/35
147/147 - 1s - loss: 0.0216 - val_loss: 0.0202
Epoch 8/35
147/147 - 2s - loss: 0.0209 - val_loss: 0.0199
Epoch 9/35
147/147 - 2s - loss: 0.0200 - val_loss: 0.0192
Epoch 10/35
147/147 - 1s - loss: 0.0195 - val_loss: 0.0188
Epoch 11/35
147/147 - 1s - loss: 0.0191 - val_loss: 0.0185
Epoch 12/35
147/147 - 1s - loss: 0.0187 - val_loss: 0.0183
Epoch 13/35
147/147 - 1s - loss: 0.0184 - val_loss: 0.0180
Epoch 14/35
147/147 - 1s - loss: 0.0181 - val_loss: 0.0178
Epoch 15/35
147/147 - 1s - loss: 0.0178 - val_loss: 0.0177
Epoch 16/35
147/147 - 2s - loss: 0.0176 - val_loss: 0.0176
Epoch 17/35
147/147 - 2s - loss: 0.0174 - val_loss: 0.0174

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=2 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


...

Fold 0
Epoch 1/35
147/147 - 2s - loss: 0.6062 - val_loss: 0.3019
Epoch 2/35
147/147 - 2s - loss: 0.1584 - val_loss: 0.0633
Epoch 3/35
147/147 - 1s - loss: 0.0477 - val_loss: 0.0354
Epoch 4/35
147/147 - 1s - loss: 0.0305 - val_loss: 0.0251
Epoch 5/35
147/147 - 2s - loss: 0.0255 - val_loss: 0.0226
Epoch 6/35
147/147 - 1s - loss: 0.0230 - val_loss: 0.0212
Epoch 7/35
147/147 - 1s - loss: 0.0215 - val_loss: 0.0199
Epoch 8/35
147/147 - 1s - loss: 0.0207 - val_loss: 0.0194
Epoch 9/35
147/147 - 1s - loss: 0.0199 - val_loss: 0.0190
Epoch 10/35
147/147 - 1s - loss: 0.0195 - val_loss: 0.0186
Epoch 11/35
147/147 - 1s - loss: 0.0192 - val_loss: 0.0184
Epoch 12/35
147/147 - 1s - loss: 0.0187 - val_loss: 0.0181
Epoch 13/35
147/147 - 1s - loss: 0.0185 - val_loss: 0.0179
Epoch 14/35
147/147 - 2s - loss: 0.0181 - val_loss: 0.0176
Epoch 15/35
147/147 - 1s - loss: 0.0179 - val_loss: 0.0175
Epoch 16/35
147/147 - 1s - loss: 0.0176 - val_loss: 0.0174
Epoch 17/35
147/147 - 2s - loss: 0.0174 - val_loss: 0.0172

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Fold 0
Epoch 1/35
147/147 - 2s - loss: 0.6049 - val_loss: 0.3084
Epoch 2/35
147/147 - 1s - loss: 0.1581 - val_loss: 0.0610
Epoch 3/35
147/147 - 2s - loss: 0.0481 - val_loss: 0.0318
Epoch 4/35
147/147 - 2s - loss: 0.0304 - val_loss: 0.0259
Epoch 5/35
147/147 - 1s - loss: 0.0252 - val_loss: 0.0229
Epoch 6/35
147/147 - 1s - loss: 0.0230 - val_loss: 0.0215
Epoch 7/35
147/147 - 1s - loss: 0.0217 - val_loss: 0.0203
Epoch 8/35
147/147 - 2s - loss: 0.0207 - val_loss: 0.0195
Epoch 9/35
147/147 - 2s - loss: 0.0200 - val_loss: 0.0192
Epoch 10/35
147/147 - 1s - loss: 0.0196 - val_loss: 0.0188
Epoch 11/35
147/147 - 1s - loss: 0.0192 - val_loss: 0.0187
Epoch 12/35
147/147 - 1s - loss: 0.0188 - val_loss: 0.0181
Epoch 13/35
147/147 - 2s - loss: 0.0185 - val_loss: 0.0179
Epoch 14/35
147/147 - 1s - loss: 0.0181 - val_loss: 0.0177
Epoch 15/35
147/147 - 1s - loss: 0.0179 - val_loss: 0.0176
Epoch 16/35
147/147 - 1s - loss: 0.0177 - val_loss: 0.0175
Epoch 17/35
147/147 - 2s - loss: 0.0174 - val_loss: 0.0171

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=6 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Fold 0
Epoch 1/35
147/147 - 2s - loss: 0.6037 - val_loss: 0.3197
Epoch 2/35
147/147 - 1s - loss: 0.1601 - val_loss: 0.0626
Epoch 3/35
147/147 - 1s - loss: 0.0476 - val_loss: 0.0331
Epoch 4/35
147/147 - 1s - loss: 0.0309 - val_loss: 0.0251
Epoch 5/35
147/147 - 2s - loss: 0.0257 - val_loss: 0.0227
Epoch 6/35
147/147 - 1s - loss: 0.0228 - val_loss: 0.0209
Epoch 7/35
147/147 - 1s - loss: 0.0217 - val_loss: 0.0202
Epoch 8/35
147/147 - 1s - loss: 0.0208 - val_loss: 0.0203
Epoch 9/35
147/147 - 1s - loss: 0.0201 - val_loss: 0.0189
Epoch 10/35
147/147 - 1s - loss: 0.0196 - val_loss: 0.0186
Epoch 11/35
147/147 - 1s - loss: 0.0191 - val_loss: 0.0183
Epoch 12/35
147/147 - 1s - loss: 0.0187 - val_loss: 0.0180
Epoch 13/35
147/147 - 2s - loss: 0.0186 - val_loss: 0.0179
Epoch 14/35
147/147 - 1s - loss: 0.0182 - val_loss: 0.0176
Epoch 15/35
147/147 - 1s - loss: 0.0179 - val_loss: 0.0175
Epoch 16/35
147/147 - 1s - loss: 0.0177 - val_loss: 0.0172
Epoch 17/35
147/147 - 2s - loss: 0.0175 - val_loss: 0.0172

In [10]:
print(f'OOF Metric: {metric(train_targets, res)}')

OOF Metric: 0.015938256498847392


In [11]:
ss.loc[test['cp_type']==1, train_targets.columns] = 0

In [12]:
ss.to_csv('submission.csv', index=False)